In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [24]:
!pip install datasets
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 3.0 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset


In [4]:
from datasets import load_dataset

dataset = load_dataset("wmt14", "de-en")  # WMT-14 English-German
train_data = dataset["train"]
valid_data = dataset["validation"]
test_data = dataset["test"]


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/280M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/265M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/273M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/474k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/509k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4508785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

In [14]:
train_data

Dataset({
    features: ['translation'],
    num_rows: 10
})

In [26]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers

# Initialize a BPE tokenizer
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Extract English and German sentences
corpus = [example["translation"]["en"] for example in train_data] + \
         [example["translation"]["de"] for example in train_data]

# Train the tokenizer on the corpus
trainer = trainers.BpeTrainer(special_tokens=["<s>", "</s>", "<pad>", "<unk>"], vocab_size=32000)
tokenizer.train_from_iterator(corpus, trainer=trainer)

# Save the tokenizer for future use
tokenizer.save("bpe_tokenizer.json")


In [28]:
def tokenize_function(example):
    # Tokenize English and German text
    src_tokens = tokenizer.encode(example["translation"]["en"]).ids
    tgt_tokens = tokenizer.encode(example["translation"]["de"]).ids
    return {"src_tokens": src_tokens, "tgt_tokens": tgt_tokens}

# Apply tokenization
train_data = train_data.map(tokenize_function, remove_columns=["translation"])
# valid_data = valid_data.map(tokenize_function, remove_columns=["translation"])
# test_data = test_data.map(tokenize_function, remove_columns=["translation"])


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [29]:
train_data

Dataset({
    features: ['src', 'tgt', 'src_tokens', 'tgt_tokens'],
    num_rows: 10
})

In [31]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

# Define a data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

# Create PyTorch DataLoaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=data_collator)
valid_loader = DataLoader(valid_data, batch_size=32, collate_fn=data_collator)
test_loader = DataLoader(test_data, batch_size=32, collate_fn=data_collator)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [32]:
train_loader

In [33]:
import torch.nn as nn

# Define the embedding layer
vocab_size = tokenizer.get_vocab_size()
d_model = 512  # Dimension of embeddings
embedding = nn.Embedding(vocab_size, d_model)

# Example usage
batch = next(iter(train_loader))  # A batch from the dataloader
src_tokens = batch["src_tokens"]  # Source token IDs
tgt_tokens = batch["tgt_tokens"]  # Target token IDs

src_embeddings = embedding(src_tokens)  # Shape: (batch_size, seq_length, d_model)
tgt_embeddings = embedding(tgt_tokens)  # Shape: (batch_size, seq_length, d_model)


AttributeError: 'tokenizers.Tokenizer' object has no attribute 'pad'

In [8]:
import torch
import numpy as np


def softmax(x: torch.Tensor, d: int):
    # return torch.nn.Softmax(dim=d)(x)
    return torch.nn.functional.softmax(x, dim=d)


class FeedForward(torch.nn.Module):
    def __init__(
        self,
        d_model: int = 512,
        hidden_size: list[tuple[int]] = None,
        activation: torch.nn.Module = torch.nn.ReLU(),
    ):
        """
        notes:
            - default dimensions are set according to paper

        """
        if hidden_size is None:
            hidden_size = [(d_model, 2048), (2048, d_model)]

        assert len(hidden_size) > 0, "hidden_size must be greater than 0"
        assert (
            hidden_size[0][0] == d_model and hidden_size[-1][1] == d_model
        ), "input and output dimensions must equal d_model"

        super().__init__()

        layers = []
        n = len(hidden_size)
        for i in range(n):
            layers.append(torch.nn.Linear(*hidden_size[i]))

            # No activation after the final layer
            if i < n - 1:
                layers.append(activation)

        self.net = torch.nn.Sequential(*layers)

    def forward(self, x: torch.Tensor):
        return self.net(x)


def attention(q: torch.Tensor, k: torch.Tensor, v: torch.Tensor, mask: bool = False):
    """
    input:
        q: a torch tensor of size: (batch_size, seq_len, d_k)
        k: a torch tensor of size: (batch_size, seq_len, d_k)
        v: a torch tensor of size: (batch_size, seq_len, d_v)
        mask: a boolean

    output:
        A torch tensor of size: (batch_size, seq_len, d_v)

    """
    x = torch.matmul(q, torch.transpose(k, 1, 2))
    d_k = torch.tensor(q.size(dim=2), dtype=torch.float32, device=x.device)

    # Scale x by sqrt(d_k)
    x = x / torch.sqrt(d_k)

    # Here we will be creating a triangular matrix where
    # all the upper triangle (above the diagonal) is set to -oo.
    if mask:
        batch_size, seq_len, _ = x.size()
        mask_x = torch.full(
            (seq_len, seq_len), float("-inf"), dtype=x.dtype, device=x.device
        )
        mask_x = torch.triu(mask_x, diagonal=1)

        # Add a new batch_size dimension and expand it to
        # match batch_size
        # -1 means keep the size at that dimension
        # However its also documented here:
        # https://stackoverflow.com/questions/65900110/does-pytorch-broadcast-consume-less-memory-than-expand
        # that expand does not also consume extra memory
        # mask_x = mask_x.unsqueeze(0).expand(batch_size, -1, -1)

        # Broadcasting will match the size at dim=0
        mask_x = mask_x.unsqueeze(0)

        # Apply mask via element-wise addition
        x = x + mask_x

    return torch.matmul(softmax(x, d=2), v)


class MultiHead(torch.nn.Module):
    def __init__(self, h: int = 8, d_model: int = 512, mask: bool = False):
        """
        input:
            h: number of heads
            d_model: model dimensions, i.e. embedding size
            mask: a boolean to apply masked multi-head attention
        notes:
            - default dimensions are set according to the paper

        """
        assert d_model % h == 0, "d_model must be divisible by h"
        super().__init__()
        self.d_model = d_model
        self.d_k = d_model // h
        self.d_v = self.d_k
        self.h = h
        self.mask = mask

        self.w_q = torch.nn.Linear(d_model, d_model)
        self.w_k = torch.nn.Linear(d_model, d_model)
        self.w_v = torch.nn.Linear(d_model, d_model)
        self.w_o = torch.nn.Linear(d_model, d_model)

    def forward(
        self,
        q: torch.Tensor,
        k: torch.Tensor,
        v: torch.Tensor,
    ):
        """
        input:
            q: a torch tensor of size: (batch_size, seq_len, d_model)
            k: a torch tensor of size: (batch_size, seq_len, d_model)
            v: a torch tensor of size: (batch_size, seq_len, d_model)

        output:
            A torch tensor of size: (batch_size, seq_len, d_model)

        notes:
            - d_model is essentially embedding dimensions
        """
        batch_size, seq_len, _ = q.size()
        q_h = self.w_q(q)
        k_h = self.w_k(k)
        v_h = self.w_v(v)

        # Splitting q, k and v tensors in to h heads
        q_h = q_h.reshape(batch_size, seq_len, self.h, self.d_k).transpose(1, 2)
        k_h = k_h.reshape(batch_size, seq_len, self.h, self.d_k).transpose(1, 2)
        v_h = v_h.reshape(batch_size, seq_len, self.h, self.d_v).transpose(1, 2)

        # Combine heads for parrallel computation
        q_h = q_h.reshape(batch_size * self.h, seq_len, self.d_k)
        k_h = k_h.reshape(batch_size * self.h, seq_len, self.d_k)
        v_h = v_h.reshape(batch_size * self.h, seq_len, self.d_v)

        # Apply attention
        scores = attention(q_h, k_h, v_h, mask=self.mask)

        # Seperate heads
        scores = scores.reshape(batch_size, self.h, seq_len, self.d_v).transpose(1, 2)

        # Concat h heads (Concat(head1, ..., headh))
        scores = scores.reshape(batch_size, seq_len, self.h * self.d_v)

        return self.w_o(scores)


# TODO: can change the definition so that if dropout called with
# p=None then can just terminate. This would make the Encoder and
# Decoder forward passes more concise
def dropout(x: torch.Tensor, p: float = 0.1, eps=1e-8):
    assert 0 <= p <= 1, "p must be a probability"

    # Create a tensor with the same shape as x
    # and set all is values to 1 - p
    mask = torch.full_like(x, 1 - p)

    # Will sample the entries from the bernoulli distribution.
    # The i'th entry of the output tensor will draw a value 1 according
    # to the i'th probability given the input tensor.
    mask = torch.bernoulli(mask).to(x.device)

    # Apply dropout via element-wise multiplication.
    x = x * mask

    # Apply inverted scaling
    return x * (1 / (1 - p + eps))


class Encoder(torch.nn.Module):
    def __init__(self, number_of_heads: int = 8, d_model: int = 512):
        """
        notes:
            - default parameter values are based on the paper
        """
        assert (
            d_model % number_of_heads == 0
        ), "d_model must be divisible by number_of_heads"
        super().__init__()
        self.multi_head_attention = MultiHead(h=number_of_heads, d_model=d_model)
        self.feed_forward = FeedForward(d_model=d_model)
        self.d_model = d_model
        self.layer_norms = torch.nn.ModuleList(
            torch.nn.LayerNorm(d_model) for _ in range(2)
        )

    def forward(self, x: torch.Tensor, dropout_p=None) -> torch.Tensor:
        """
        input:
            x: a torch tensor of size: (batch_size, seq_len, d_model)
        output:
            a torch tensor of size: (batch_size, seq_len, d_model)
        """

        x = self.layer_norms[0](
            x
            + (
                self.multi_head_attention(x, x, x)
                if dropout_p is None
                else dropout(self.multi_head_attention(x, x, x), p=dropout_p)
            )
        )

        return self.layer_norms[1](
            x
            + (
                self.feed_forward(x)
                if dropout_p is None
                else dropout(self.feed_forward(x), p=dropout_p)
            )
        )


class Decoder(torch.nn.Module):
    def __init__(self, number_of_heads: int = 8, d_model: int = 512):
        """
        notes:
            - default parameter values are based on the paper
            - in the previous implementation we were using the same multi head
            module and inidicating the apply mask on the forward pass. That's an
            outregous mistake. The same parameters are being used in that case
            only when applying attention part of the sequence was getting masked.
        """
        assert (
            d_model % number_of_heads == 0
        ), "d_model must be divisible by number_of_heads"
        super().__init__()
        self.multi_head_attention = MultiHead(h=number_of_heads, d_model=d_model)
        self.masked_multi_head_attention = MultiHead(
            h=number_of_heads, d_model=d_model, mask=True
        )
        self.feed_forward = FeedForward(d_model=d_model)
        self.d_model = d_model
        self.layer_norms = torch.nn.ModuleList(
            torch.nn.LayerNorm(d_model) for _ in range(3)
        )

    def forward(
        self, x: torch.Tensor, encoder_x: torch.Tensor, dropout_p=None
    ) -> torch.Tensor:
        """
        inputs:
            x: a torch tensor of size: (batch_size, seq_len, d_model)
            encoder_x: a torch tensor of size: (batch_size, seq_len, d_model)
        output:
            a torch tensor of size: (batch_size, seq_len, d_model)
        """
        x = self.layer_norms[0](
            x
            + (
                self.masked_multi_head_attention(x, x, x)
                if dropout_p is None
                else dropout(self.masked_multi_head_attention(x, x, x), p=dropout_p)
            )
        )

        x = self.layer_norms[1](
            x
            + (
                self.multi_head_attention(x, encoder_x, encoder_x)
                if dropout_p is None
                else dropout(
                    self.multi_head_attention(x, encoder_x, encoder_x), p=dropout_p
                )
            )
        )

        return self.layer_norms[2](
            x
            + (
                self.feed_forward(x)
                if dropout_p is None
                else dropout(self.feed_forward(x), p=dropout_p)
            )
        )


def positional_encodings(seq_len: int, d_model: int) -> np.ndarray:
    pos, i = np.indices((seq_len, d_model))
    return np.where(
        i % 2 == 0,
        np.sin(pos / np.power(10000, (2 * i / d_model))),
        np.cos(pos / np.power(10000, (2 * i / d_model))),
    )


class Transformer(torch.nn.Module):
    def __init__(
        self,
        batch_size: int,
        seq_len: int,
        d_model: int = 512,
        n: int = 6,
        number_of_heads: int = 8,
        vocab_size: int = 37000,
    ):
        """
        inputs:
            n: the number of encoder and decoder stacks
            d_model: model dimensions, i.e. embedding dimensions
            number_of_heads: number of heads using in multi-head attention
            vocab_size: number of tokens in the vocabulary

        """
        super().__init__()
        self.n = n
        self.d_model = d_model
        self.number_of_heads = number_of_heads
        self.vocab_size = vocab_size
        self.register_buffer(
            "pe",
            torch.from_numpy(positional_encodings(seq_len, d_model))
            .to(torch.float32)
            .unsqueeze(0),
        )
        self.linear = torch.nn.Linear(d_model, vocab_size)

        # Using ModuleList is crucial here instead of python list
        # If python list is used, then model.parameters() will not
        # return the paremeters of layers inside.
        self.encoders = torch.nn.ModuleList(
            Encoder(number_of_heads=number_of_heads, d_model=d_model)
            for _ in range(self.n)
        )
        self.decoders = torch.nn.ModuleList(
            Decoder(number_of_heads=number_of_heads, d_model=d_model)
            for _ in range(self.n)
        )

    def forward(self, encoder_x: torch.Tensor, decoder_x: torch.Tensor):
        """
        inputs:
            encoder_x: a torch tensor of size: (batch_size, seq_len, d_model)
            decoder_x: a torch tensor of size: (batch_size, seq_len, d_model)

        outputs:
            a probability distribution over the vocabulary
        """

        # TODO: adding positional encodings at each pass for testing now,
        # I don't think pe has to be added at each pass.
        encoder_x = encoder_x + self.pe
        decoder_x = decoder_x + self.pe

        for encoder in self.encoders:
            encoder_x = encoder(encoder_x)

        for decoder in self.decoders:
            decoder_x = decoder(decoder_x, encoder_x)

        return softmax(self.linear(decoder_x), d=2)

In [9]:
import torch
import numpy as np
from torch.utils.data import DataLoader

# Use the CountingDataset
class CountingDataset(torch.utils.data.Dataset):
    def __init__(self, n, max_length=8, vocab_size=8):
        assert vocab_size > 2
        self.n = n
        self.vocab_size = vocab_size
        seq_lengths = np.random.randint(max_length // 2, max_length, n)
        data = np.random.randint(0, vocab_size, [n, max_length])

        # Replace elements past the sequence length with -1
        for i in range(n):
            data[i, slice(seq_lengths[i] + 1, None)] = -1

        # Convert to one-hot
        onehot_data = np.zeros([n, max_length, vocab_size])
        for v in range(vocab_size):
            onehot_row = np.zeros([vocab_size])
            onehot_row[v] = 1
            onehot_data[data == v] = onehot_row

        # Label is whether ones outnumber twos in the sequence
        num_ones = (data == 1).sum(axis=1, keepdims=True)
        num_twos = (data == 2).sum(axis=1, keepdims=True)
        label = (num_ones > num_twos).astype(int)

        self.data = torch.tensor(onehot_data).float()
        self.label = torch.tensor(label).long()

    def __len__(self):
        return self.n

    def __getitem__(self, item_index):
        return self.data[item_index], self.label[item_index]


# Create dataset and dataloader
dataset = CountingDataset(1000, max_length=8, vocab_size=3)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

vocab_size = 3  # Matches the CountingDataset
d_model = 3     # Set d_model to match vocab_size

# Transformer Model Parameters
batch_size = 32
seq_len = 8
d_model = vocab_size  # Match d_model with vocab_size
number_of_heads = 1   # Reduce number of heads since d_model=3
n = 2


# Instantiate Transformer Model
model = Transformer(
    batch_size=batch_size,
    seq_len=seq_len,
    d_model=d_model,
    n=n,
    number_of_heads=number_of_heads,
    vocab_size=vocab_size
)

# Define Loss and Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 120
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for x, y in dataloader:
        # Forward pass
        encoder_x = x  # Encoder input
        decoder_x = torch.zeros_like(x)  # Decoder input as zero (start-of-sequence token)
        output = model(encoder_x, decoder_x)

        # Reshape for loss calculation
        output = output[:, -1, :]  # Take the output for the last token
        y = y.squeeze()  # Squeeze labels for compatibility

        # Calculate loss
        loss = criterion(output, y)
        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader)}")

# Testing the Model
model.eval()
with torch.no_grad():
    test_data, test_labels = next(iter(dataloader))
    encoder_x = test_data
    decoder_x = torch.zeros_like(test_data)
    output = model(encoder_x, decoder_x)
    predictions = torch.argmax(output[:, -1, :], dim=-1)

    print("Test Labels:", test_labels.squeeze().numpy())
    print("Predictions:", predictions.numpy())


Epoch 1/120, Loss: 1.001364465802908
Epoch 2/120, Loss: 0.9847359694540501
Epoch 3/120, Loss: 0.9738593213260174
Epoch 4/120, Loss: 0.9625486638396978
Epoch 5/120, Loss: 0.9545099586248398
Epoch 6/120, Loss: 0.9529047086834908
Epoch 7/120, Loss: 0.9496243707835674
Epoch 8/120, Loss: 0.9414796400815248
Epoch 9/120, Loss: 0.9466585628688335
Epoch 10/120, Loss: 0.9430395718663931
Epoch 11/120, Loss: 0.9420137908309698
Epoch 12/120, Loss: 0.9440227523446083
Epoch 13/120, Loss: 0.941102122887969
Epoch 14/120, Loss: 0.940063115209341
Epoch 15/120, Loss: 0.9375380575656891
Epoch 16/120, Loss: 0.9365776423364878
Epoch 17/120, Loss: 0.9360838457942009
Epoch 18/120, Loss: 0.935663603246212
Epoch 19/120, Loss: 0.9317116513848305
Epoch 20/120, Loss: 0.9327497761696577
Epoch 21/120, Loss: 0.9400257524102926
Epoch 22/120, Loss: 0.9356538895517588
Epoch 23/120, Loss: 0.9320124946534634
Epoch 24/120, Loss: 0.9348272997885942
Epoch 25/120, Loss: 0.9346113614737988
Epoch 26/120, Loss: 0.9376609195023775